In [1]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_7d5d0d16a6454b0987a63e487c91c25b.csv"
df=pd.read_csv(url)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
df.shape

(768, 9)

In [3]:
df.isna().any()

Pregnancies                 False
Glucose                     False
BloodPressure               False
SkinThickness               False
Insulin                     False
BMI                         False
DiabetesPedigreeFunction    False
Age                         False
Outcome                     False
dtype: bool

In [4]:
df.groupby("Outcome").count().T

Outcome,0,1
Pregnancies,500,268
Glucose,500,268
BloodPressure,500,268
SkinThickness,500,268
Insulin,500,268
BMI,500,268
DiabetesPedigreeFunction,500,268
Age,500,268


In [5]:
X=df.drop(["Outcome"],axis=1)
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [6]:
y=df[["Outcome"]]
y.head()

,Outcome
0,1
1,0
2,1
3,0
4,1


In [7]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496


In [8]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

# feature extraction. We are going to output the selection scores for all features and select the features with the highest scores.
test = SelectKBest(score_func=mutual_info_classif, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
1,0.126471
5,0.073039
4,0.054468
7,0.046709
0,0.030666
2,0.012276
6,0.011371
3,0.009957


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
print(X_train.shape)
print(X_test.shape)

(652, 8)
(116, 8)


In [10]:
from sklearn.ensemble import GradientBoostingClassifier
gradient=GradientBoostingClassifier(loss="deviance",learning_rate=0.01,n_estimators=100,subsample=1,criterion="mse",min_samples_split=3,min_samples_leaf=2,max_depth=2,max_features="auto",validation_fraction=0.2,random_state=0).fit(X_train,y_train)
gradient

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='mse', init=None, learning_rate=0.01,
              loss='deviance', max_depth=2, max_features='auto',
              max_leaf_nodes=None, min_impurity_decrease=0.0,
              min_impurity_split=None, min_samples_leaf=2,
              min_samples_split=3, min_weight_fraction_leaf=0.0,
              n_estimators=100, n_iter_no_change=None, presort='auto',
              random_state=0, subsample=1, tol=0.0001,
              validation_fraction=0.2, verbose=0, warm_start=False)

In [11]:
from sklearn.metrics import accuracy_score
y0_pred=gradient.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=gradient.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.7622699386503068
Accuracy for test data is:  0.75


In [13]:
from sklearn.metrics import roc_auc_score
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.6752937768950678
AUC for test data is:  0.6848684210526317


In [14]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[409  15]
 [140  88]]
Confusion matrix for test data is: 
 [[68  8]
 [21 19]]


In [15]:
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(n_estimators=300,criterion="gini",max_depth=4,min_samples_split=3,min_samples_leaf=3,max_features="auto",bootstrap=True,n_jobs=-1,warm_start=True,class_weight="balanced_subsample",random_state=0).fit(X_train,y_train)
forest

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:502: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or "balanced_subsample" are '


RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=4, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=3,
            min_samples_split=3, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=-1, oob_score=False, random_state=0,
            verbose=0, warm_start=True)

In [16]:
y0_pred=forest.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=forest.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.8251533742331288
Accuracy for test data is:  0.7672413793103449


In [17]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.8280577623303542
AUC for test data is:  0.780921052631579


In [18]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[347  77]
 [ 37 191]]
Confusion matrix for test data is: 
 [[56 20]
 [ 7 33]]


In [19]:
from sklearn.naive_bayes import GaussianNB
NB=GaussianNB().fit(X_train,y_train)
NB

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None, var_smoothing=1e-09)

In [20]:
y0_pred=NB.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=NB.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.7530674846625767
Accuracy for test data is:  0.7844827586206896


In [21]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.7168776895067859
AUC for test data is:  0.7763157894736842


In [22]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[355  69]
 [ 92 136]]
Confusion matrix for test data is: 
 [[61 15]
 [10 30]]


In [23]:
from sklearn.gaussian_process import GaussianProcessClassifier
Gaussian=GaussianProcessClassifier(n_restarts_optimizer=3,max_iter_predict=300,warm_start=True,multi_class="one_vs_one",n_jobs=-1,random_state=0).fit(X_train,y_train)
Gaussian

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianProcessClassifier(copy_X_train=True, kernel=None,
             max_iter_predict=300, multi_class='one_vs_one', n_jobs=-1,
             n_restarts_optimizer=3, optimizer='fmin_l_bfgs_b',
             random_state=0, warm_start=True)

In [24]:
y0_pred=Gaussian.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=Gaussian.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.8880368098159509
Accuracy for test data is:  0.6724137931034483


In [25]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.8561320754716982
AUC for test data is:  0.6493421052631578


In [26]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[408  16]
 [ 57 171]]
Confusion matrix for test data is: 
 [[55 21]
 [17 23]]


In [27]:
from sklearn.linear_model import LogisticRegression
logistic=LogisticRegression(penalty="l2",C=0.1,class_weight="balanced",solver="saga",max_iter=200,multi_class="multinomial",n_jobs=-1,random_state=0).fit(X_train,y_train)
logistic

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=0.1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=200,
          multi_class='multinomial', n_jobs=-1, penalty='l2',
          random_state=0, solver='saga', tol=0.0001, verbose=0,
          warm_start=False)

In [28]:
y0_pred=logistic.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=logistic.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.7622699386503068
Accuracy for test data is:  0.7068965517241379


In [29]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.7584202250910295
AUC for test data is:  0.7171052631578948


In [30]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[327  97]
 [ 58 170]]
Confusion matrix for test data is: 
 [[52 24]
 [10 30]]


In [31]:
from sklearn import svm
svm=svm.SVC(C=0.2,kernel="poly",degree=3,gamma="scale",cache_size=200,decision_function_shape="ovo",random_state=0).fit(X_train,y_train)
svm

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=0.2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='scale', kernel='poly',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [32]:
y0_pred=svm.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=svm.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.7622699386503068
Accuracy for test data is:  0.7586206896551724


In [33]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.6742800397219464
AUC for test data is:  0.6796052631578947


In [34]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[410  14]
 [141  87]]
Confusion matrix for test data is: 
 [[71  5]
 [23 17]]


In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam

In [12]:
length=X_test.shape[1]
length

8

In [13]:
from keras.utils import to_categorical
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
num_classes=y_test.shape[1]
print(num_classes)

2


In [23]:
def classification_model():
    # create model
    model = Sequential()
    model.add(Dense(length, activation='relu', input_shape=(length,)))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.33))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.33))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.33))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    opt=Adam(lr=0.01)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [24]:
model=classification_model()
model.fit(X_train,y_train,validation_data=(X_test, y_test),epochs=10, verbose=2)
model.evaluate(X_test,y_test,verbose=0)

Train on 652 samples, validate on 116 samples
Epoch 1/10
 - 2s - loss: 0.6183 - acc: 0.7086 - val_loss: 0.5671 - val_acc: 0.6207
Epoch 2/10
 - 1s - loss: 0.5191 - acc: 0.7592 - val_loss: 0.5185 - val_acc: 0.7586
Epoch 3/10
 - 1s - loss: 0.5379 - acc: 0.7408 - val_loss: 0.5711 - val_acc: 0.7759
Epoch 4/10
 - 1s - loss: 0.4973 - acc: 0.7745 - val_loss: 0.5394 - val_acc: 0.7500
Epoch 5/10
 - 1s - loss: 0.4783 - acc: 0.7592 - val_loss: 0.5330 - val_acc: 0.7931
Epoch 6/10
 - 1s - loss: 0.4832 - acc: 0.7730 - val_loss: 0.5456 - val_acc: 0.8103
Epoch 7/10
 - 1s - loss: 0.4763 - acc: 0.7653 - val_loss: 0.5206 - val_acc: 0.7931
Epoch 8/10
 - 1s - loss: 0.4538 - acc: 0.7853 - val_loss: 0.5461 - val_acc: 0.7845
Epoch 9/10
 - 1s - loss: 0.4705 - acc: 0.7837 - val_loss: 0.5698 - val_acc: 0.7500
Epoch 10/10
 - 1s - loss: 0.4564 - acc: 0.7960 - val_loss: 0.5542 - val_acc: 0.7672


[0.5541516275241457, 0.7672413772550123]

In [34]:
116*0.7672

88.9952

In [35]:
116-89

27

In [36]:
from sklearn.metrics import roc_auc_score
y0_pred=model.predict(X_train)
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
y1_pred=model.predict(X_test)
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.8787342767295598
AUC for test data is:  0.8266447368421053


In [37]:
y1_pred=round(pd.DataFrame(y1_pred).rename(columns={0:"Pred_0",1:"Pred_1"}),1)
y1_pred.head()

,Pred_0,Pred_1
0,0.6,0.4
1,1.0,0.0
2,1.0,0.0
3,0.7,0.3
4,0.7,0.3


In [38]:
y1_pred["Pred_0"]=np.where((y1_pred.Pred_0 >=0.5),1,y1_pred.Pred_0)
y1_pred["Pred_0"]=np.where((y1_pred.Pred_0 < 0.5),0,y1_pred.Pred_0)
y1_pred["Pred_1"]=np.where((y1_pred.Pred_1 >=0.5),1,y1_pred.Pred_1)
y1_pred["Pred_1"]=np.where((y1_pred.Pred_1 <0.5),0,y1_pred.Pred_1)
y1_pred.head()

,Pred_0,Pred_1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [39]:
y1_pred["Pred_0"]=np.where((y1_pred.Pred_0 == y1_pred.Pred_1),0,y1_pred.Pred_0)
y1_pred.head()

,Pred_0,Pred_1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [40]:
y_test=pd.DataFrame(y_test).rename(columns={0:"Actual_0",1:"Actual_1"})
y_test.head()

,Actual_0,Actual_1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [41]:
len0=len(y_test[y_test["Actual_0"]==1])
print("The number of actual negative cases: ",len0)

The number of actual negative cases:  76


In [42]:
len1=len(y_test[y_test["Actual_1"]==1])
print("The number of actual positive cases: ",len1)

The number of actual positive cases:  40


In [43]:
len00=len(y1_pred[y1_pred["Pred_0"]==1])
print("The number of predicted negative cases: ",len00)

The number of predicted negative cases:  68


In [44]:
len11=len(y1_pred[y1_pred["Pred_1"]==1])
print("The number of predicted positive cases: ",len11)

The number of predicted positive cases:  48


In [45]:
print("Confusion matrix on test data: \n",np.array([[59,17],[9,31]]))

Confusion matrix on test data: 
 [[59 17]
 [ 9 31]]
